In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [3]:
house_address_all = []
house_details_all = []
house_address = []
house_details = []
postal_code_array = []
latitude_array = []
longitude_array = []
year_built_array = []
payment_monthly = []

base_url = 'https://www.remax.ca/ab/calgary-real-estate?page='
urls = [base_url + str(x) for x in range(1,5)]

for url in urls:
    
    #browser.visit(url)
    time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    time.sleep(5)

    try:
        addresses = soup.find_all('div', class_='left-content flex-one')
        for address in addresses:
            house_address_all.append(address.text)
    except:
        house_address_all.append('None')
        
    time.sleep(5)
        
    try:
        details = soup.find_all('div', class_='property-details')
        for detail in details:
            house_details_all.append(detail.text)
    except:
        house_details_all.append('None')
        
    time.sleep(5)
        
    house_details_more=[]
    url_house = []
        
    try:
        for detail in soup.find_all('a'):
            house_details_more.append(detail.get('href'))
    except:
        house_details_more.append('None')
    
    time.sleep(5)

    for i in house_details_more:
        if '/ab/calgary-real-estate/' in str(i):
            link = 'https://www.remax.ca' + str(i)
            url_house.append(link)
            
    time.sleep(5)
    
    for i in url_house:
    
        time.sleep(5)

        page = requests.get(i)
        soup = BeautifulSoup(page.text, "html.parser")

        lat_all = soup.find_all('script')[4]

        string = ', '.join(lat_all)
        pc = "postalCode"
        position_pc = string.find(pc)
        pcd = string[(position_pc+14):(position_pc+21)]
        postal_code_array.append(pcd)

        lat = "latitude"
        position_lat = string.find(lat)
        latd = string[(position_lat+11):(position_lat+20)]
        latitude_array.append(latd)

        long = "longitude"
        position_long = string.find(long)
        longd = string[(position_long+12):(position_long+23)]
        longitude_array.append(longd)

        house_address = []

        time.sleep(5)

        try:
            addresses = soup.find_all('section', class_='property-details')
            for address in addresses:
                house_address.append(address.text)
        except:
            house_address.append('None')

        time.sleep(5)

        string = ', '.join(house_address)
        s_year = "Year Built"
        position_year = string.find(s_year)
        year_built = string[(position_year+12):(position_year+16)]
        year_built_array.append(year_built)
        
        try:
            payment = soup.find(class_="payment-amount").text
            payment_monthly.append(payment)
        except:
            payment_monthly.append('None')

        time.sleep(5)

In [4]:
#postal_code_array[:5]
#latitude_array[:5]
#longitude_array[:5]
#year_built_array[:5]

In [5]:
#soup.find(class_="payment-amount").text

In [62]:
address_df = pd.DataFrame(house_address_all)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")
new_df["price"] = new_df["price"].str.replace(",", "")
new_df["price"] = pd.to_numeric(new_df["price"])

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,"4312 49 ST NE, Calgary, AB, T1Y 2Z3",329900
1,"108 TUSCANY VALLEY DR NW, Calgary, AB, T3L 2E7",425000
2,"2107 - 450 SAGE VALLEY DR NW, Calgary, AB, T3R...",234900
3,"6852 RUNDLEHORN DR NE, Calgary, AB, T1Y 3V2",369000
4,"27 MIDRIDGE CLOSE SE, Calgary, AB, T2X 1G2",424900


In [63]:
final_df_pc = new_df[2].str.split(', Calgary, AB, ', expand=True)
final_df_pc.head()

,0,1
0,4312 49 ST NE,T1Y 2Z3
1,108 TUSCANY VALLEY DR NW,T3L 2E7
2,2107 - 450 SAGE VALLEY DR NW,T3R 0V5
3,6852 RUNDLEHORN DR NE,T1Y 3V2
4,27 MIDRIDGE CLOSE SE,T2X 1G2


In [64]:
final_df_pc_clean = final_df_pc[1].str.replace(" ", "")
final_df_pc_clean.head()

0    T1Y2Z3
1    T3L2E7
2    T3R0V5
3    T1Y3V2
4    T2X1G2
Name: 1, dtype: object

In [65]:
df_add = pd.concat([new_df, final_df_pc_clean, final_df_pc[0]], axis=1)
del df_add[2]
df_add.columns = ["price", "postal_code", "address"]
df_add.head()

,price,postal_code,address
0,329900,T1Y2Z3,4312 49 ST NE
1,425000,T3L2E7,108 TUSCANY VALLEY DR NW
2,234900,T3R0V5,2107 - 450 SAGE VALLEY DR NW
3,369000,T1Y3V2,6852 RUNDLEHORN DR NE
4,424900,T2X1G2,27 MIDRIDGE CLOSE SE


In [66]:
details = pd.DataFrame(house_details_all)
details_df_temp = details[0].str.split('|', expand=True)
details_df_temp.head()

,0,1,2,3
0,4 bed,1 + 1 bath,2300 sqft,house
1,4 bed,2 + 1 bath,1336 sqft,house
2,2 bed,2 bath,835 sqft,condo
3,5 bed,3 bath,1136 sqft,house
4,3 bed,2 bath,1055 sqft,house


In [67]:
details_df_bed = details_df_temp[0].str.replace(' bed', '')
details_df_bath = details_df_temp[1].str.replace(' bath', '')
details_df_area = details_df_temp[2].str.replace(' sqft', '')

In [68]:
details_df_bath_all = details_df_bath.str.split('+', expand=True)
details_df_bath_full = details_df_bath_all[0]
details_df_bath_half = details_df_bath_all[1]

In [69]:
details_df_bed = details_df_bed.replace('N/A', np.nan)
details_df_bed = pd.to_numeric(details_df_bed)
details_df_area = details_df_area.replace('N/A', np.nan)
details_df_area = pd.to_numeric(details_df_area)
details_df_bath_full = details_df_bath_full.replace('N/A', np.nan)
details_df_bath_full = pd.to_numeric(details_df_bath_full)
details_df_bath_half = details_df_bath_half.replace('N/A', np.nan)
details_df_bath_half = pd.to_numeric(details_df_bath_half)

In [70]:
data = {'bed':details_df_bed, 'full_bath':details_df_bath_full, 'half_bath':details_df_bath_half,
       'property_area':details_df_area, 'property_type':details_df_temp[3]}

In [71]:
details_df = pd.DataFrame(data)
details_df.head()

,bed,full_bath,half_bath,property_area,property_type
0,4.0,1.0,1.0,2300.0,house
1,4.0,2.0,1.0,1336.0,house
2,2.0,2.0,NaN,835.0,condo
3,5.0,3.0,NaN,1136.0,house
4,3.0,2.0,NaN,1055.0,house


In [72]:
calgary_df_dup = pd.concat([df_add, details_df], axis=1)
calgary_df = calgary_df_dup.drop_duplicates()
calgary_df.head()

,price,postal_code,address,bed,full_bath,half_bath,property_area,property_type
0,329900,T1Y2Z3,4312 49 ST NE,4.0,1.0,1.0,2300.0,house
1,425000,T3L2E7,108 TUSCANY VALLEY DR NW,4.0,2.0,1.0,1336.0,house
2,234900,T3R0V5,2107 - 450 SAGE VALLEY DR NW,2.0,2.0,NaN,835.0,condo
3,369000,T1Y3V2,6852 RUNDLEHORN DR NE,5.0,3.0,NaN,1136.0,house
4,424900,T2X1G2,27 MIDRIDGE CLOSE SE,3.0,2.0,NaN,1055.0,house


## Cleanup Postal Codes, Latitude, Longitude and Year Built

In [73]:
postal_codes_messy = pd.DataFrame(postal_code_array, columns=["postal_code"])

In [74]:
postal_codes_messy.head()

,postal_code
0,"T1Y2Z3"""
1,"T3L2E7"""
2,"T3R0V5"""
3,"T1Y3V2"""
4,"T2X1G2"""


In [75]:
postal_code_array_clean = postal_codes_messy["postal_code"].str.replace('"', '')
postal_code_array_clean = postal_code_array_clean.str.replace(" ", "")

In [76]:
latitude_array_messy = pd.DataFrame(latitude_array, columns=["latitude"])

In [77]:
latitude_array_messy = latitude_array_messy["latitude"].str.replace(',', '')
latitude_array_messy.head()

0    51.090065
1    51.128175
2    51.179434
3     51.07927
4     50.92003
Name: latitude, dtype: object

In [78]:
longitude_array_messy = pd.DataFrame(longitude_array, columns=["longitude"])

In [79]:
longitude_array_messy = longitude_array_messy["longitude"].str.replace(',', '')
longitude_array_messy.head()

0     -113.962366
1     -114.247503
2     -114.158282
3     -113.940553
4    -114.05498\n
Name: longitude, dtype: object

<div class="alert alert-danger">
    <b>Unused</b>
    
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])<br>
year_built_clean = year_built_messy["year_built"].str.replace('', np.nan)<br>
year_built_num = year_built_messy.astype(int)

year_built_num.head()
</div>

In [80]:
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])
#year_built_messy = pd.to_numeric(year_built_messy)
year_built_messy.head()

,year_built
0,1977
1,1998
2,2015
3,1978
4,1978


In [81]:
year_built_clean = year_built_messy["year_built"].replace('', np.nan)

In [82]:
year_built_clean.head(10)

0    1977
1    1998
2    2015
3    1978
4    1978
5    1976
6    2004
7    nasq
8    1960
9    2005
Name: year_built, dtype: object

In [83]:
year_built_digits = []

for i in year_built_clean:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        #year_built_clean.replace(i, np.nan)
        year_built_digits.append(np.nan)
    else:
        year_built_digits.append(i)

In [84]:
year_built_digits[:10]

['1977', '1998', '2015', '1978', '1978', '1976', '2004', nan, '1960', '2005']

In [85]:
year_built_num = pd.to_numeric(year_built_digits)
year_built_num[:10]

array([1977., 1998., 2015., 1978., 1978., 1976., 2004.,   nan, 1960.,
       2005.])

In [86]:
years_old = 2020 - year_built_num
years_old[:10]

array([43., 22.,  5., 42., 42., 44., 16., nan, 60., 15.])

In [87]:
latitude_digits = []

for i in latitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        latitude_digits.append(np.nan)
    else:
        latitude_digits.append(i)

In [89]:
longitude_digits = []

for i in longitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        longitude_digits.append(np.nan)
    else:
        longitude_digits.append(i)

In [90]:
latitude_digits = pd.to_numeric(latitude_digits)
longitude_digits = pd.to_numeric(longitude_digits)

In [91]:
payment_monthly_messy = pd.DataFrame(payment_monthly, columns=["monthly_payments"])

In [92]:
payment_monthly_messy = payment_monthly_messy["monthly_payments"].str.replace('$', '')
payment_monthly_messy.head()

0    1426.53
1    1837.75
2    1015.74
3    1595.60
4    1837.32
Name: monthly_payments, dtype: object

In [93]:
payment_monthly_clean = payment_monthly_messy.replace('None', np.nan)
payment_monthly_clean = pd.to_numeric(payment_monthly_clean)

<div class="alert alert-success">
    <b>Guide</b>
    
https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

#### approximate radius of earth in km
R = 6373.0
    
lat1 = radians(52.2296756)<br>
lon1 = radians(21.0122287)

lat2 = radians(52.406374)<br>
lon2 = radians(16.9251681)

dlon = lon2 - lon1<br>
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2<br>
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
    
#### Calgary Downtown coordinates (51.0448304,-114.0715378)
</div>

In [97]:
distance_downtown = []

for i, j in zip(latitude_digits, longitude_digits):
    lat1 = radians(i)
    lon1 = radians(j)
    lat2 = radians(-114.0715378)
    lon2 = radians(51.0448304)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6373.0
    distance = (R * c)/1000
    
    distance_downtown.append(distance)

In [98]:
distance_downtown[:5]

[13.078496922312384,
 13.080357678425717,
 13.086749595863642,
 13.077486305291568,
 13.058948693555799]

In [99]:
distance_downtown_df = pd.DataFrame(distance_downtown)

<div class="alert alert-warning">
    <b>Another example</b>
    
import geopy.distance

coords_1 = (52.2296756, 21.0122287)<br>
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)
</div>

In [100]:
more_details = pd.DataFrame([latitude_digits, longitude_digits, year_built_num]).T
#more_details.columns = ["postal_code", "latitude", "longitude", "year_built"]
more_details.head()

,0,1,2
0,51.090065,-113.962366,1977.0
1,51.128175,-114.247503,1998.0
2,51.179434,-114.158282,2015.0
3,51.079270,-113.940553,1978.0
4,50.920030,-114.054980,1978.0


In [101]:
more_df = pd.concat([postal_code_array_clean, more_details, years_old, distance_downtown_df, payment_monthly_clean], axis=1)
more_df.columns = ["postal_code", "latitude", "longitude", "year_built", "years_old", "distance_downtown", "monthly_payments"]
more_df.head()

TypeError: cannot concatenate object of type "<class 'numpy.ndarray'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [40]:
result_dup = pd.merge(calgary_df, more_df, on='postal_code')
result = result_dup.drop_duplicates()
result = result.reset_index(drop=True)
result.head()

,price,postal_code,address,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments
0,329900,T1Y2Z3,4312 49 ST NE,4,1,1.0,2300,house,51.090065,-113.962366,1977.0,43.0,18.354762,1426.53
1,464900,T3A0M1,3916 VANCOUVER CRES NW,4,2,NaN,1043,house,51.091708,-114.143333,1964.0,56.0,18.354943,2010.28
2,285000,T2Z3V4,97 Prestwick Gardens SE,2,2,NaN,1120,townhouse,50.917976,-113.962580,1999.0,21.0,18.335740,1232.38
3,1196000,T2N2Z6,707 35 ST NW,4,2,1.0,2155,house,51.061022,-114.138279,NaN,NaN,18.351552,NaN
4,614900,T3M0S3,63 CRANARCH CIR SE,4,3,1.0,2210,house,50.878539,-113.978402,2011.0,9.0,18.331381,2658.90


In [ ]:
result.to_csv('data/calgary_listings.csv', index=False)

-----------

<div class="alert alert-info">
    <b>UNSED</b>
</div>

In [ ]:
house_details_more=[]
url_house = []
try:
    for detail in soup.find_all('a'):
        house_details_more.append(detail.get('href'))
except:
    house_details_more.append('None')
    
for i in house_details_more:
    if '/ab/calgary-real-estate/' in str(i):
        link = 'https://www.remax.ca' + str(i)
        url_house.append(link)
url_house

In [ ]:
url_house[:5]

In [ ]:
PostalCode = []
Latitude = []
Longitude = []
YearBuilt = []

for i in url_house[:5]:
    
    time.sleep(5)
    
    page = requests.get(i)
    soup = BeautifulSoup(page.text, "html.parser")
    
    lat_all = soup.find_all('script')[4]
    
    string = ', '.join(lat_all)
    pc = "postalCode"
    position_pc = string.find(pc)
    pcd = string[(position_pc+14):(position_pc+21)]
    PostalCode.append(pcd)

    lat = "latitude"
    position_lat = string.find(lat)
    latd = string[(position_lat+11):(position_lat+20)]
    Latitude.append(latd)

    long = "longitude"
    position_long = string.find(long)
    longd = string[(position_long+12):(position_long+23)]
    Longitude.append(longd)
    
    house_address = []
    
    time.sleep(5)
    
    try:
        addresses = soup.find_all('section', class_='property-details')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
    
    time.sleep(5)
    
    string = ', '.join(house_address)
    s_year = "Year Built"
    position_year = string.find(s_year)
    year_built = string[(position_year+12):(position_year+16)]
    YearBuilt.append(year_built)